# DCGAN with MNIST

* MNIST data를 가지고 **Deep Convolution GAN**를 `tf.contrib.slim`을 이용하여 만들어보자.
  * [참고: TensorFlow slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)

## Import modules

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)

#tf.set_random_seed(219)
#np.random.seed(219)

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/Users/shatapy/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [15]:
# Training Flags (hyperparameter configuration)
train_dir = 'train/dcgan/exp1/'
max_epochs = 30
save_epochs = 10
summary_steps = 2500
print_steps = 500
batch_size = 64
learning_rate_D = 0.0002
learning_rate_G = 0.001
k = 1 # the number of step of learning D before learning G
num_samples = 16

## Import MNIST

In [3]:
# Load training and eval data from tf.keras
(train_data, train_labels), _ = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data
train_data = train_data / 255.
train_labels = np.asarray(train_labels, dtype=np.int32)

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [4]:
# for train
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = train_dataset.shuffle(buffer_size = 10000)
train_dataset = train_dataset.repeat(count=max_epochs)
train_dataset = train_dataset.batch(batch_size = batch_size)
print(train_dataset)

<BatchDataset shapes: (?, 28, 28), types: tf.float64>


## Create the model

`conv2d_transpose` => fractional-strided convolution to increase the size of generated images. 

`padding='SAME' => OUTPUT_HEIGHT = INPUT_HEIGHT / STRIDE  
conv2d_transpose & padding='SAME' => OUTPUT_HEIGHT = INPUT_HEIGHT * STRIDE`

<strong>"Directly applying batchnorm to all layers
however, resulted in sample oscillation and model instability. This was avoided by not applying
batchnorm to the generator output layer and the discriminator input layer."</strong> from DCGAN paper

<strong>"The first layer of the GAN, which takes a uniform noise distribution Z as input, could be called
    fully connected as it is just a matrix multiplication, but the result is reshaped into a 4-dimensional
    tensor and used as the start of the convolution stack."

In [5]:
class DCGAN(object):
  """Deep Convolutional Generative Adversarial Networks
  implementation based on http://arxiv.org/abs/1511.06434
  
  "Unsupervised Representation Learning with
  Deep Convolutional Generative Adversarial Networks"
  Alec Radford, Luke Metz and Soumith Chintala
  """
  
  def __init__(self, mode, train_dataset, test_dataset=None):
    """Basic setup.
    
    Args:
      mode (`string`): "train" or "generate".
      train_dataset (`tf.data.Dataset`): train_dataset.
      test_dataset (`tf.data.Dataset`): test_dataset.
    """
    assert mode in ["train", "generate"]
    self.mode = mode
    
    # hyper-parameters for model
    self.x_dim = 28
    self.z_dim = 100
    self.batch_size = batch_size
    self.num_samples = num_samples
    self.train_dataset = train_dataset
    self.test_dataset = test_dataset
    
    # Global step Tensor.
    self.global_step = None
    
    print('The mode is %s.' % self.mode)
    print('complete initializing model.')
    
    
  def build_random_z_inputs(self):
    """Build a vector random_z in latent space.
    
    Returns:
      self.random_z (`2-rank Tensor` with [batch_size, z_dim]):
          latent vector which size is generally 100 dim.
      self.sample_random_z (`2-rank Tensor` with [num_samples, z_dim]):
          latent vector which size is generally 100 dim.
 
    "The first layer of the GAN, which takes a uniform noise distribution Z as input, could be called
    fully connected as it is just a matrix multiplication, but the result is reshaped into a 4-dimensional
    tensor and used as the start of the convolution stack."
    """
    
    with tf.variable_scope('random_z'):
      self.random_z = tf.random_uniform(shape=[self.batch_size, 1, 1, self.z_dim],  
                                        minval=-1.0, maxval=1.0)
      self.sample_random_z = tf.random_uniform(shape=[self.num_samples, 1, 1, self.z_dim],
                                               minval=-1.0, maxval=1.0)

    return self.random_z, self.sample_random_z
  
  
  def read_MNIST(self, dataset):
    """Read MNIST dataset
    
    Args:
      dataset (`tf.data.Dataset` format): MNIST dataset.
      
    Returns:
      self.mnist (`4-rank Tensor` with [batch, x_dim, x_dim, 1]): MNIST dataset with batch size.
    """
    with tf.variable_scope('mnist'):
      iterator = dataset.make_one_shot_iterator()

      self.mnist = iterator.get_next()
      self.mnist = tf.cast(self.mnist, dtype = tf.float32)
      self.mnist = tf.expand_dims(self.mnist, axis=3)
      
    return self.mnist


  def Generator(self, random_z, is_training=True, reuse=False):
    """Generator setup.
    
    Args:
      random_z (`2-rank Tensor` with [batch_size, z_dim]):
          latent vector which size is generally 100 dim.
      is_training (`bool`): whether training mode or test mode.
      reuse (`bool`): whether variable reuse or not.
      
    Returns:
      generated_data (`4-rank Tensor` with [batch_size, h, w, c])
          generated images from random vector z.
    """
    with tf.variable_scope('Generator', reuse=reuse) as scope:
      batch_norm_params = {'decay': 0.9,
                           'epsilon': 0.001,
                           'is_training': is_training,
                           'scope': 'batch_norm'}
      with slim.arg_scope([slim.conv2d_transpose],
                          kernel_size=[4, 4],
                          stride=[2, 2],
                          normalizer_fn=slim.batch_norm,
                          normalizer_params=batch_norm_params):
        # 여기를 채워 넣으세요
        # Use full conv2d_transpose instead of projection and reshape
        # conv2d_transpose = fractional-strided convolution to increase the size of generated images. 
        # random_z: 1 x 1 x 100 dim
        self.inputs = random_z
        # inputs = random_z: 1 x 1 x 100 dim
        # outputs: 3 x 3 x 256 dim
        # OH = stride * (IH - 1) + FH - 2 * Padding
        self.layer1 = slim.conv2d_transpose(self.inputs, 
                                            num_outputs=256,
                                            kernel_size=[3, 3], 
                                            padding='VALID')  
        # inputs: 3 x 3 x 256 dim
        # outputs: 7 x 7 x 128 dim
        self.layer2 = slim.conv2d_transpose(self.layer1,
                                           num_outputs=128,
                                           kernel_size=[3, 3],
                                           padding='VALID')
        # inputs: 7 x 7 x 128 dim
        # outputs: 14 x 14 x 64 dim
        self.layer3 = slim.conv2d_transpose(self.layer2,
                                           num_outputs=64)
        
        # inputs: 14 x 14 x 64 dim
        # outputs: 28 x 28 x 1 dim
        self.layer4 = slim.conv2d_transpose(self.layer3,
                                           num_outputs=1,
                                           normalizer_fn=None,
                                           activation_fn=tf.tanh)
        # generated_data = outputs: 28 x 28 x 1 dim
        generated_data = self.layer4

        return generated_data
    
    
  def Discriminator(self, data, reuse=False):
    """Discriminator setup.
    
    Args:
      data (`2-rank Tensor` with [batch_size, x_dim]): MNIST real data.
      reuse (`bool`): whether variable reuse or not.
      
    Returns:
      logits (`1-rank Tensor` with [batch_size]): logits of data.
    """
    with tf.variable_scope('Discriminator', reuse=reuse) as scope:
      batch_norm_params = {'decay': 0.9,
                           'epsilon': 0.001,
                           'scope': 'batch_norm'}
      with slim.arg_scope([slim.conv2d],
                          kernel_size=[4, 4],
                          stride=[2, 2],
                          activation_fn=tf.nn.leaky_relu,
                          normalizer_fn=slim.batch_norm,
                          normalizer_params=batch_norm_params):
        # 여기를 채워 넣으세요
        # inputs = data: 28 x 28 x 1 dim
        # outputs: 14 x 14 x 64 dim
        self.layer1 = slim.conv2d(data, 64, normalizer_fn=None)
        # inputs: 14 x 14 x 64 dim
        # outputs: 7 x 7 x 128 dim
        self.layer2 = slim.conv2d(self.layer1, 128)
        # inputs: 7 x 7 x 128 dim
        # outputs: 3 x 3 x 256 dim
        # self.layer3 = slim.conv2d(self.layer2, 256)
        self.layer3 = slim.conv2d(self.layer2, 256, kernel_size=[3, 3], padding='VALID')
        # inputs: 3 x 3 x 256 dim
        # outputs: 1 x 1 x 1 dim
        self.layer4 = slim.conv2d(self.layer3, 1, kernel_size=[3, 3], 
                                  stride=[1, 1], 
                                  padding='VALID',
                                  normalizer_fn=None,
                                  activation_fn=None)
        # logits = layer4: 1 x 1 x 1 dim -> 1 dim
        discriminator_logits = tf.squeeze(self.layer4, axis=[1,2])

        return discriminator_logits
    
    
  def setup_global_step(self):
    """Sets up the global step Tensor."""
    if self.mode == "train":
      self.global_step = tf.train.get_or_create_global_step()
      
      print('complete setup global_step.')
      
      
  def GANLoss(self, logits, is_real=True, scope=None):
    """Computes standard GAN loss between `logits` and `labels`.
    
    Args:
      logits (`1-rank Tensor`): logits.
      is_real (`bool`): True means `1` labeling, False means `0` labeling.
      
    Returns:
      loss (`0-randk Tensor): the standard GAN loss value. (binary_cross_entropy)
    """
    if is_real:
      labels = tf.ones_like(logits)
    else:
      labels = tf.zeros_like(logits)

    # 여기를 채워 넣으세요
    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels,
                                          logits=logits, scope=scope)
    #sigmoid x_ent = mostly for binary classification.

    return loss

      
  def build(self):
    """Creates all ops for training or generate."""
    self.setup_global_step()
    
    if self.mode == "generate":
      pass
    
    else:
      # generating random vector
      self.random_z, self.sample_random_z = self.build_random_z_inputs()
      # read dataset
      self.real_data = self.read_MNIST(self.train_dataset)
      
      # 여기를 채워 넣으세요
      # generating images from Generator() via random vector z
      self.generated_data = self.Generator(self.random_z)
      
      # 여기를 채워 넣으세요
      # discriminating real data by Discriminator()
      self.real_logits = self.Discriminator(self.real_data)
      # discriminating fake data (generated)_images) by Discriminator()
      self.fake_logits = self.Discriminator(self.generated_data, reuse=True)
      
      # 여기를 채워 넣으세요
      # losses of real with label "1"
      self.loss_real = self.GANLoss(self.real_logits)
      # losses of fake with label "0"
      self.loss_fake = self.GANLoss(self.fake_logits, is_real=False)
      
      # losses of Discriminator
      with tf.variable_scope('loss_D'):
        self.loss_Discriminator = self.loss_real + self.loss_fake
        
      # 여기를 채워 넣으세요
      # losses of Generator with label "1" that used to fool the Discriminator
      self.loss_Generator = self.GANLoss(self.fake_logits)
      
      # 여기를 채워 넣으세요
      # Separate variables for each function
      self.D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Discriminator')
      self.G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Generator')
      
      
      # generating images for sample
      self.sample_data = self.Generator(self.sample_random_z, is_training=False, reuse=True)
      
      # write summaries
      # Add loss summaries
      tf.summary.scalar('losses/loss_Discriminator', self.loss_Discriminator)
      tf.summary.scalar('losses/loss_Generator', self.loss_Generator)
      
      # Add histogram summaries
      for var in self.D_vars:
        tf.summary.histogram(var.op.name, var)
      for var in self.G_vars:
        tf.summary.histogram(var.op.name, var)
      
      # Add image summaries
      tf.summary.image('random_images', self.generated_data, max_outputs=4)
      tf.summary.image('real_images', self.real_data)
      
    print('complete model build.\n')

## Define plot function

In [6]:
def print_sample_data(sample_data, max_print=num_samples):
  print_images = sample_data[:max_print,:]
  print_images = print_images.reshape([max_print, 28, 28])
  print_images = print_images.swapaxes(0, 1)
  print_images = print_images.reshape([28, max_print * 28])
    
  print("printing")
  
  plt.figure(figsize=(max_print, 1))
  plt.axis('off')
  plt.imshow(print_images, cmap='gray')
  plt.show()

## Build a model

In [7]:
model = DCGAN(mode="train", train_dataset=train_dataset)
model.build()

# # show info for trainable variables
# t_vars = tf.trainable_variables()
# slim.model_analyzer.analyze_vars(t_vars, print_info=True)

The mode is train.
complete initializing model.
complete setup global_step.
complete model build.



In [8]:
opt_D = tf.train.AdamOptimizer(learning_rate=learning_rate_D, beta1=0.5)
opt_G = tf.train.AdamOptimizer(learning_rate=learning_rate_G, beta1=0.5)

### Batch Normalization

In [9]:
D_collection = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='Discriminator')
G_collection = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='Generator')

with tf.control_dependencies(D_collection):
  opt_D_op = opt_D.minimize(model.loss_Discriminator, var_list=model.D_vars)
with tf.control_dependencies(G_collection):
  opt_G_op = opt_G.minimize(model.loss_Generator, global_step=model.global_step, var_list=model.G_vars)

### Assign `tf.summary.FileWriter`

In [10]:
graph_location = train_dir
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph()) 

Saving graph to: train/dcgan/exp1/


### `tf.summary`

In [11]:
summary_op = tf.summary.merge_all()

### `tf.train.Saver`

In [12]:
saver = tf.train.Saver(tf.global_variables(), max_to_keep=1000)

### `tf.Session` and train

In [17]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
  sess.run(tf.global_variables_initializer())
  tf.logging.info('Start Session.')
  
  num_examples = len(train_data)
  num_batches_per_epoch = int(num_examples / batch_size)
  
  # save loss values for plot
  loss_history = []
  pre_epochs = 0
  while True:
    try:
      start_time = time.time()
      
      for _ in range(k):
        _, loss_D = sess.run([opt_D_op, model.loss_Discriminator])
      _, global_step_, loss_G = sess.run([opt_G_op,
                                          model.global_step,
                                          model.loss_Generator])
      
      epochs = global_step_ * batch_size / float(num_examples)
      duration = time.time() - start_time

      if global_step_ % print_steps == 0:
        examples_per_sec = batch_size / float(duration)
        print("Epochs: {:.2f} global_step: {} loss_D: {:.3f} loss_G: {:.3f} ({:.2f} examples/sec; {:.3f} sec/batch)".format(
                  epochs, global_step_, loss_D, loss_G, examples_per_sec, duration))

        loss_history.append([epochs, loss_D, loss_G])

        # print sample data
        sample_data = sess.run(model.sample_data)
        print_sample_data(sample_data)

      # write summaries periodically
      if global_step_ % summary_steps == 0:
        summary_str = sess.run(summary_op)
        train_writer.add_summary(summary_str, global_step=global_step_)

      # save model checkpoint periodically
      if int(epochs) % save_epochs == 0  and  pre_epochs != int(epochs):
        tf.logging.info('Saving model with global step {} (= {} epochs) to disk.'.format(global_step_, int(epochs)))
        saver.save(sess, train_dir + 'model.ckpt', global_step=global_step_)
        pre_epochs = int(epochs)
        
    except tf.errors.OutOfRangeError:
      print("End of dataset")  # ==> "End of dataset"
      tf.logging.info('Saving model with global step {} (= {} epochs) to disk.'.format(global_step_, int(epochs)))
      saver.save(sess, train_dir + 'model.ckpt', global_step=global_step_)
      break
      
  tf.logging.info('complete training...')

KeyboardInterrupt: 

## Plot loss functions

In [ ]:
loss_history = np.asarray(loss_history)

plt.plot(loss_history[:,0], loss_history[:,1], label='loss_D')
plt.plot(loss_history[:,0], loss_history[:,2], label='loss_G')
plt.legend(loc='upper right')
plt.show()